# Train Diffusion Policy for SO-101 Robot

This notebook trains a **Diffusion Policy** on the SO-101 robot arm for pick-and-place tasks using [LeRobot](https://github.com/huggingface/lerobot).

## What is Diffusion Policy?
Diffusion Policy is a state-of-the-art imitation learning method that uses diffusion models to generate robot actions. It has shown excellent performance on manipulation tasks.

## Dataset
We use the official `lerobot/svla_so101_pickplace` dataset which contains 50 episodes (11,939 frames) of pick-and-place demonstrations on the SO-101 arm.

## Requirements
- Google Colab with GPU runtime (T4 or better)
- Hugging Face account for uploading trained models

## 1. Install Dependencies

In [ ]:
!pip install lerobot

## 2. Login to Hugging Face

You'll need a Hugging Face account to upload your trained model. Get your token from https://huggingface.co/settings/tokens

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## 3. Set Environment Variables

Replace `your_username` with your actual Hugging Face username.

In [ ]:
import os

# Replace with your Hugging Face username
HF_USER = "your_username"  # @param {type:"string"}

os.environ["HF_USER"] = HF_USER
print(f"Hugging Face user set to: {HF_USER}")

## 4. Verify GPU Availability

In [ ]:
import torch

if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("WARNING: No GPU detected! Go to Runtime > Change runtime type > GPU")

## 5. Train Diffusion Policy on SO-101 Pick-Place Dataset

This will train a Diffusion Policy on the official SO-101 pick-place dataset.

**Training parameters:**
- `batch_size=32`: Reduced from 64 due to Colab memory limits
- `steps=100000`: Good starting point for training
- `eval_freq=10000`: Evaluate every 10k steps
- `save_freq=10000`: Save checkpoint every 10k steps

In [ ]:
!python -m lerobot.scripts.train \
    --policy.type=diffusion \
    --dataset.repo_id=lerobot/svla_so101_pickplace \
    --output_dir=outputs/train/diffusion_so101_pickplace \
    --job_name=diffusion_so101_pickplace \
    --policy.device=cuda \
    --batch_size=32 \
    --steps=100000 \
    --eval_freq=10000 \
    --save_freq=10000 \
    --wandb.enable=false

## 6. (Optional) Train on Your Own Dataset

If you have recorded your own SO-101 dataset, you can train on it instead.

First, record a dataset following the [LeRobot data collection guide](https://github.com/huggingface/lerobot/blob/main/examples/7_get_started_with_real_robot.md), then run:

In [ ]:
# Uncomment and modify to train on your own dataset

# CUSTOM_DATASET = f"{HF_USER}/your_so101_dataset"  # Replace with your dataset

# !python -m lerobot.scripts.train \
#     --policy.type=diffusion \
#     --dataset.repo_id={CUSTOM_DATASET} \
#     --output_dir=outputs/train/diffusion_so101_custom \
#     --job_name=diffusion_so101_custom \
#     --policy.device=cuda \
#     --batch_size=32 \
#     --steps=100000 \
#     --eval_freq=10000 \
#     --save_freq=10000 \
#     --wandb.enable=false

## 7. Upload Trained Model to Hugging Face Hub

After training completes, upload your model to share it or use it for inference on your robot.

In [ ]:
import os

HF_USER = os.environ.get("HF_USER", "your_username")
MODEL_NAME = "diffusion_so101_pickplace"
CHECKPOINT_PATH = "outputs/train/diffusion_so101_pickplace/checkpoints/last/pretrained_model"

print(f"Uploading model to: {HF_USER}/{MODEL_NAME}")
!huggingface-cli upload {HF_USER}/{MODEL_NAME} {CHECKPOINT_PATH}

## 8. Test Inference (Optional)

You can test the trained policy with a quick inference check.

In [ ]:
from lerobot.common.policies.diffusion.modeling_diffusion import DiffusionPolicy
from lerobot.common.policies.diffusion.configuration_diffusion import DiffusionConfig
import torch

# Load the trained model
checkpoint_path = "outputs/train/diffusion_so101_pickplace/checkpoints/last/pretrained_model"

try:
    policy = DiffusionPolicy.from_pretrained(checkpoint_path)
    policy.eval()
    print("Model loaded successfully!")
    print(f"Policy config: {policy.config}")
except Exception as e:
    print(f"Could not load model: {e}")
    print("This is expected if training hasn't completed yet.")

## Next Steps

1. **Deploy to your robot**: Download the model from Hugging Face Hub and run inference on your SO-101
2. **Fine-tune**: If performance isn't satisfactory, collect more demonstrations and continue training
3. **Compare policies**: Try training ACT or other policies on the same dataset to compare performance

### Running inference on your robot:

```python
from lerobot.common.policies.diffusion.modeling_diffusion import DiffusionPolicy

# Load from Hub
policy = DiffusionPolicy.from_pretrained("your_username/diffusion_so101_pickplace")
policy.eval()

# Run inference
# ... (integrate with your robot control code)
```

## References

- [LeRobot GitHub](https://github.com/huggingface/lerobot)
- [Diffusion Policy Paper](https://arxiv.org/abs/2303.04137)
- [SO-101 Pick-Place Dataset](https://huggingface.co/datasets/lerobot/svla_so101_pickplace)
- [LeRobot Documentation](https://huggingface.co/docs/lerobot)